# Definición de bilbiotecas necesarias

In [2]:
import pandas as pd
import numpy as np
import random
import string
from faker import Faker

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Creación del dataset

In [45]:
# Definir el número de registros
n_registros = 10000

# Generar datos aleatorios para las columnas
np.random.seed(42)  # Para reproducibilidad

# 1. id: identificadores únicos
id = [f"ID{''.join(random.choices(string.ascii_uppercase, k=3))}{''.join(random.choices(string.digits, k=4))}" for _ in range(n_registros)]

# 2. nombre: Generador de nombres aleatorios
fake = Faker('es_MX')
nombres = [fake.name() for _ in range(n_registros)]

# 3. semestre (faltante): Semestre que cursan con valores faltantes aleatorios (convertimos a float para manejar NaN)
semestre = np.random.randint(1, 11, size=n_registros)#.astype(float)
#semestre[np.random.rand(n_registros) < 0.2] = np.nan  # 20% de datos faltantes

# 4. promedio (completo): Promedio entre 60 y 100
promedio = np.random.randint(60, 101, size=n_registros)

# 5. ingresos: Ingresos mensuales: Generar salarios entre 8300 y 100,000
ingresos = np.array([round(random.gauss(20000, 9000)) for _ in range(n_registros)]).astype(float)
ingresos = np.clip(ingresos, 8300, None)
ingresos[np.random.rand(n_registros) < 0.2] = np.nan  # 20% de datos faltantes

# 6. opinion_campus (faltante): Opinión del estudiante sobre su campus del 1 al 10 con 20% de datos faltantes (convertimos a float para manejar NaN)
opinion_campus = np.array([round(random.gauss(8, 4)) for _ in range(n_registros)]).astype(int)
opinion_campus = np.clip(opinion_campus, 1, 10)


# 7. trabaja: Si el estudiante trabaja (completo): falso o positivo
trabaja = np.array([round(random.gauss(0.1, 0.4)) for _ in range(n_registros)]).astype(int)
trabaja = list(map(bool, trabaja))

# 8. distancia_universidad: Distancia en kilómetros entre 5 y 50
distancia_universidad = np.array([round(random.gauss(3, 5)) for _ in range(n_registros)]).astype(float)
distancia_universidad = np.clip(distancia_universidad, 1, None)
distancia_universidad[np.random.rand(n_registros) < 0.2] = np.nan  # 20% de datos faltantes

# 9. edad: Edad entre 17 y 60
edad = np.array([round(random.gauss(20, 5)) for _ in range(n_registros)]).astype(int)
edad = np.clip(edad, 17, None)

# 10. tipo_vivienda: Propia o Alquiler (categórico)
tipo_vivienda = np.random.choice(['Propia', 'Alquiler'], size=n_registros)

# 12. horas_estudio: Horas dedicadas al mes a estudiar. Variable con dos clusters a los lados y media en el centro
data_1_30 = np.random.randint(1, 31, int(n_registros/2))
data_70_100 = np.random.randint(70, 101, int(n_registros/2))
horas_estudio = np.concatenate([data_1_30, data_70_100])

# 12. desertor: Se generará de forma que dependa de ciertas características.
# Relacionamos la clase objetivo con ciertas variables
desertor = np.zeros(n_registros)

# Se activa si:
# - ingresos < 9000
# - promedio < 65
# - distancia_universidad > 15
# - semestre < 3
# - opinion_campus < 5
desertor[((ingresos < 9000) | (promedio < 65) | (distancia_universidad > 10) | (horas_estudio < 5)) & ((semestre < 3) | (opinion_campus < 5))] = 1

# Introducir ruido: aleatoriamente cambiar algunos valores de la clase objetivo para que no sea 100% separable.
ruido_indices = np.random.choice(n_registros, size=int(n_registros * 0.1), replace=False)  # 10% de ruido
desertor[ruido_indices] = 1 - desertor[ruido_indices]  # Invertir valores en los índices de ruido
desertor = desertor.astype(int)

# Crear el DataFrame
df = pd.DataFrame({
    'id': id,
    'nombre': nombres,
    'semestre': semestre,
    'promedio': promedio,
    'ingresos': ingresos,
    'opinion_campus': opinion_campus,
    'trabaja': trabaja,
    'distancia_universidad': distancia_universidad,
    'edad': edad,
    'tipo_vivienda': tipo_vivienda,
    'horas_estudio': horas_estudio,
    'desertor': desertor
})

df


,id,nombre,semestre,promedio,ingresos,opinion_campus,trabaja,distancia_universidad,edad,tipo_vivienda,horas_estudio,desertor
0,IDHFI3552,Caridad Leonor Padrón Cavazos,7,86,13580.0,10,False,NaN,21,Propia,29,0
1,IDVOH5097,Aurelio Fabiola Anguiano,4,89,15993.0,4,True,1.0,22,Alquiler,5,0
2,IDFYS5079,Omar Amalia Sepúlveda,8,89,11008.0,4,False,1.0,18,Propia,11,0
3,IDSCE4123,Pamela Aida Cuellar Valverde,5,62,30926.0,10,False,1.0,17,Propia,12,0
4,IDYMU9480,Óscar Olivárez Nájera,7,66,24968.0,10,False,1.0,26,Alquiler,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,IDURC8308,Inés Cynthia Madrid Salinas,7,86,17337.0,10,True,1.0,37,Alquiler,70,0
9996,IDJLX9083,Mitzy Puente,10,74,NaN,1,False,10.0,17,Alquiler,88,0
9997,IDNCQ0036,Raquel Longoria,3,91,26994.0,10,True,NaN,27,Propia,71,0
9998,IDWJQ8803,Lic. Andrea Cisneros,10,87,NaN,7,True,3.0,26,Alquiler,74,0


# EDA del dataset para confirmar comportameinto

In [46]:
df.describe()

,semestre,promedio,ingresos,opinion_campus,distancia_universidad,edad,horas_estudio,desertor
count,10000.000000,10000.000000,8107.000000,10000.000000,7977.000000,10000.000000,10000.000000,10000.000000
mean,5.497200,79.752400,20319.281485,7.271800,4.264385,20.888600,50.270400,0.187600
std,2.893964,11.743368,8143.080849,2.833254,3.632390,3.855733,35.786128,0.390412
min,1.000000,60.000000,8300.000000,1.000000,1.000000,17.000000,1.000000,0.000000
25%,3.000000,70.000000,14035.000000,5.000000,1.000000,17.000000,16.000000,0.000000
50%,6.000000,80.000000,19972.000000,8.000000,3.000000,20.000000,50.000000,0.000000
75%,8.000000,90.000000,25908.500000,10.000000,7.000000,23.000000,85.000000,0.000000
max,10.000000,100.000000,49329.000000,10.000000,25.000000,39.000000,100.000000,1.000000


In [47]:
df['desertor'].value_counts()

desertor
0    8124
1    1876
Name: count, dtype: int64

# Clasificación del dataset para verificar comportameinto

In [42]:
X = df.drop(columns=['id', 'nombre', 'tipo_vivienda', 'desertor'])
y = df['desertor'].astype(int)

In [43]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.8980
Confusion Matrix:
[[2385   30]
 [ 276  309]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      2415
           1       0.91      0.53      0.67       585

    accuracy                           0.90      3000
   macro avg       0.90      0.76      0.80      3000
weighted avg       0.90      0.90      0.89      3000



# Guardar el dataset

In [49]:
df.to_csv('dataset.csv',index=False)